## Importation des packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from ydata_profiling import ProfileReport
import missingno as msno #msno.matrix(df)
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectFromModel
import pygwalker as pyg

## Importation des fichiers CSV

In [ ]:
edl_es = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\Fichier CSV\EDL Entrant avec EDL sortant corresp Constat.csv", sep=';')
edl_patrimoine = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\Fichier CSV\EDL_0 Sortants_Patrimoines_Locataires.csv", sep=';', low_memory=False)
edl_sollicitation = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\Fichier CSV\EDL_1 Sollicitations GRC.csv", sep=';')
edl_enquete = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\Fichier CSV\EDL_3 Frais Non-Rep Enquetes.csv", sep=';')
edl_ost = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\Fichier CSV\EDL_4 Labels OST Non Valeur.csv", sep=';')
edl_travaux = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\Fichier CSV\EDL_Nb contrats_ Mt Factures Travaux - V2.csv", sep=';')

# EDA

## EDL_ES = Etat des lieux Entrant Sortant

In [ ]:
edl_es

dimension du DataFrame:

In [ ]:
edl_es.shape

type des colonnes:

In [ ]:
edl_es.info()

nombre de valeurs manquante:

In [ ]:
edl_es.isna().sum()

affichage graphique des valeurs manquantes:

In [ ]:
msno.matrix(edl_es)

## EDL_patrimoine = Patrimoines des locataires sortant

In [ ]:
edl_patrimoine

dimension du DataFrame:

In [ ]:
edl_patrimoine.shape

nombre de valeurs manquantes

In [ ]:
edl_patrimoine.isna().sum()

type de colonnes:

In [ ]:
edl_patrimoine.info()

affichage graphique des valeurs manquantes 

In [ ]:
msno.matrix(edl_patrimoine)

## EDL_sollicitation = description des travaux/entretiens pour un logement, demander par le locataire 

In [ ]:
edl_sollicitation

dimension du DataFrame:

In [ ]:
edl_sollicitation.shape

type des colonnes:

In [ ]:
edl_sollicitation.info()

affichage des valeurs manquantes:

In [ ]:
edl_sollicitation.isna().sum()

affichage graphique des valeurs manquantes:

In [ ]:
msno.matrix(edl_sollicitation)

## EDL_enquete = frais de non reponses aux enquetes 

In [ ]:
edl_enquete

dimension du DataFrame:

In [ ]:
edl_enquete.shape

type des colonnes:

In [ ]:
edl_enquete.info()

affichage des valeurs manquantes:

In [ ]:
edl_enquete.isna().sum()

affichage graphique des valeurs manquantes:

In [ ]:
msno.matrix(edl_enquete)

## EDL_OST = Occupant Sans droit ni Titre

In [ ]:
edl_ost


dimension du DataFrame:

In [ ]:
edl_ost.shape

type des colonnes:

In [ ]:
edl_ost.info()

affichage des valeurs manquantes:

In [ ]:
edl_ost.isna().sum()

affichage graphique des valeurs manquantes:

In [ ]:
msno.matrix(edl_ost)

## EDL_travaux = montant des travaux totaux par logement 

In [ ]:
edl_travaux

dimension du DataFrame:

In [ ]:
edl_travaux.shape

type des colonnes:

In [ ]:
edl_travaux.info()

affichage des valeurs manquantes:

In [ ]:
edl_travaux.isna().sum()

affichage graphique des valeurs manquantes:

In [ ]:
msno.matrix(edl_travaux)

# Data Cleaning

## EDL_ES = Etat des lieux Entrant Sortant

Conversion des colonnes cible en dates:

In [ ]:
edl_es['DTD_CTRAT'] = pd.to_datetime(edl_es['DTD_CTRAT'])
edl_es['DTF_CTRAT'] = pd.to_datetime(edl_es['DTF_CTRAT'])
edl_es['DT_DOSEDL_ENT'] = pd.to_datetime(edl_es['DT_DOSEDL_ENT'])
edl_es['DT_DOSEDL_SOR'] = pd.to_datetime(edl_es['DT_DOSEDL_SOR'])

Liste des tache:

In [ ]:
# concatenation des 3 colonnes: CD_PATRIM, NO_CTRACLI, NO_EDL_ENT
# affichier en double la colonne CD_PATRIM pour effectuer les jointures
# drop colonnes des colonnes: ID_PERS_ENT, ID_PERS_SOR
# mise en place du barem pour les colonnes: LB_NOTSANT_ENT, LB_NOTSANT_SOR
# ajouter une colonnes pour avoir la note global a l'entré et a la sortie
# creer un dataframe d'etat des lieux sortant et un dataframe ou les etats des lieux n'ont pas était défini

Creation du DataFrame de test:

In [ ]:
test_edl_es  = edl_es.copy()

Creation de la cle primaire:

In [ ]:
test_edl_es['cle_primaire'] = test_edl_es[['CD_PATRIM', 'NO_CTRACLI', 'NO_EDL_ENT']].astype(str).agg('-'.join, axis=1)

Drop des colonnes non pertinente:

In [ ]:
test_edl_es.drop(columns = ['ID_PERS_ENT', 'ID_PERS_SOR', 'NO_EDL_ENT'], inplace = True)

Affichage des valeurs des colonnes cible pour le barem:

In [ ]:
test_edl_es['LB_NOTSANT_ENT'].unique()

In [ ]:
test_edl_es['LB_NOTSANT_SOR'].unique()

Mise en place du barem:

In [ ]:
mapping = {
    'NEUF                          ': 100,
    'BON ETAT                      ': 85,
    'USURE NORM                    ': 75,
    'VETUSTE                       ': 35,
    'ENDOMMAGE                     ': 15,
    'DETERIORE                     ': 5,
    'MANQUANT                      ': 0,
    'EN FONCTION                   ': 0,
    'HORS SERVICE                  ': 0
}

In [ ]:
test_edl_es['LB_NOTSANT_ENT'] = test_edl_es['LB_NOTSANT_ENT'].map(mapping)

In [ ]:
test_edl_es['LB_NOTSANT_SOR'] = test_edl_es['LB_NOTSANT_SOR'].map(mapping)

Creation de la note global a l'entree et a la sortie:

In [ ]:
edl_entree_globale = pd.DataFrame(test_edl_es.groupby('cle_primaire')['LB_NOTSANT_ENT'].sum())
edl_entree_globale.reset_index(inplace = True)

In [ ]:
edl_entree_globale['cle_primaire'].nunique()

In [ ]:
edl_sortant_globale = pd.DataFrame(test_edl_es.groupby('cle_primaire')['LB_NOTSANT_SOR'].sum())
edl_sortant_globale.reset_index(inplace = True)

In [ ]:
edl_sortant_globale['cle_primaire'].nunique()

creation du dataframe avec les EDL entrant sortant:

In [ ]:
edl_sortant = test_edl_es.dropna()
edl_sortant

In [ ]:
edl_sortant = edl_sortant.merge(edl_entree_globale, on = 'cle_primaire', how = 'inner')

In [ ]:
edl_sortant = edl_sortant.merge(edl_sortant_globale, on = 'cle_primaire', how = 'inner')

In [ ]:
edl_sortant['cle_primaire'].nunique()

In [ ]:
edl_sortant.shape

In [ ]:
edl_entree_sortie = edl_sortant.drop_duplicates(subset = 'cle_primaire')
edl_entree_sortie

creation du dataframe avec que les entre:

In [ ]:
test_edl_es.shape

In [ ]:
test_edl_es['cle_jointure'] = test_edl_es[['CD_PATRIM', 'NO_CTRACLI']].astype(str).agg('-'.join, axis=1)

table pour tester notre formule d'hypotese

In [ ]:
edl_entree = test_edl_es[test_edl_es['CD_TYPEDL_SOR'].isna()]
edl_entree.shape

In [ ]:
edl_entree = edl_entree.merge(edl_entree_globale, on='cle_primaire', how='inner')

In [ ]:
edl_entree = edl_entree.drop_duplicates(subset = 'cle_primaire')

## edl_entree = Table des EDL entrant sans les sortie (locataire toujours dans le logement)

cette table servira de test pour nos hypothèse

In [ ]:
edl_entree

In [ ]:
edl_entree.info()

Drop des colonnes non pertinente:

In [ ]:
edl_entree.drop(columns = [
    'Unnamed: 0', 'NO_EDL_SOR',	'NO_CONSEDL_SOR', 
    'CD_TYPEDL_SOR', 'DT_DOSEDL_SOR', 'LB_NOTSANT_SOR', 
    'DT_DOSEDL_ENT', 'LB_NOTSANT_ENT_x', 'LB_CONTENU1', 
    'LB_CONTENU2', 'DTF_CTRAT', 'NO_CONSEDL_ENT'
    ], inplace = True)

Changement de l'ordre des colonnes:

In [ ]:
edl_entree = edl_entree.reindex(['cle_primaire', 'CD_PATRIM', 'NO_CTRACLI', 'DTD_CTRAT', 'CD_TYPEDL_ENT', 'LB_NOTSANT_ENT_y'], axis = 1)

Changement de nom de la dernière colonne:

In [ ]:
edl_entree.rename(columns = {'LB_NOTSANT_ENT_y' : 'LB_NOTSANT_ENT'}, inplace = True)

In [ ]:
edl_entree

Concatenation CD_PATRIM et numero client. Qui est notre cle pour les jointures:

In [ ]:
edl_entree['cle_jointure'] = edl_entree[['CD_PATRIM', 'NO_CTRACLI']].astype(str).agg('-'.join, axis=1)

In [ ]:
edl_entree.to_csv('edl_entree.csv', index = False)

## edl_entree_sortie = Table des EDL d'entre et de sortie

Ceci est notre base EDL pour faire nos analyse pour le sujet A (pour les prévisite des logements).
C'est sur cette table que seras incrémenter nos autre DataFrame.

In [ ]:
edl_entree_sortie

Drop des colonnes non pertinente:

In [ ]:
edl_entree_sortie.drop(columns = [
    'Unnamed: 0', 'NO_CONSEDL_ENT', 'DT_DOSEDL_ENT', 
    'NO_EDL_SOR', 'NO_CONSEDL_SOR', 'DT_DOSEDL_SOR', 
    'LB_CONTENU1', 'LB_CONTENU2', 'LB_NOTSANT_ENT_x', 
    'LB_NOTSANT_SOR_x'], inplace = True)

In [ ]:
edl_entree_sortie

Changement de l'ordre des colonnes:

In [ ]:
edl_entree_sortie = edl_entree_sortie.reindex(['cle_primaire', 'CD_PATRIM','NO_CTRACLI', 'DTD_CTRAT','DTF_CTRAT', 'CD_TYPEDL_ENT', 'CD_TYPEDL_SOR', 'LB_NOTSANT_ENT_y', 'LB_NOTSANT_SOR_y'], axis = 1)

Changement des noms des colonnes cible:

In [ ]:
edl_entree_sortie.rename(columns = {'LB_NOTSANT_ENT_y' : 'LB_NOTSANT_ENT', 'LB_NOTSANT_SOR_y' : 'LB_NOTSANT_SOR'}, inplace = True)

In [ ]:
edl_entree_sortie

Creation de la cle de jointure:

In [ ]:
edl_entree_sortie['cle_jointure'] = edl_entree_sortie[['CD_PATRIM', 'NO_CTRACLI']].astype(str).agg('-'.join, axis=1)

## EDL_enquete = frais de non reponses aux enquetes 

Changement des colonnes cible en Datetime:

In [ ]:
edl_enquete['DTD_CTRAT'] = pd.to_datetime(edl_enquete['DTD_CTRAT'], format='%Y-%m-%d')
edl_enquete['DTF_CTRAT'] = pd.to_datetime(edl_enquete['DTF_CTRAT'], format='%Y-%m-%d')
edl_enquete['DEBUT_FRAIS_DT_SID_FACTURA_GIM'] = pd.to_datetime(edl_enquete['DEBUT_FRAIS_DT_SID_FACTURA_GIM'], format='%Y-%m-%d')
edl_enquete['FIN_FRAIS_DT_SID_FACTURA_GIM'] = pd.to_datetime(edl_enquete['FIN_FRAIS_DT_SID_FACTURA_GIM'], format='%Y-%m-%d')

Changement des colonnes cible en Float:

In [ ]:
edl_enquete['MT_CPT_PENALITE RETARD_SLS_EOS'] = edl_enquete['MT_CPT_PENALITE RETARD_SLS_EOS'].str.replace(',', '.').astype(float)
edl_enquete['MT_FRAIS DOSS_SLS_EOS'] = edl_enquete['MT_FRAIS DOSS_SLS_EOS'].str.replace(',', '.').astype(float)

Changement des colonnes cible en STR:

In [ ]:
edl_enquete['PERS PHYSIQUE_MORALE'] = edl_enquete['PERS PHYSIQUE_MORALE'].astype(str)
edl_enquete['CD_PATRIM'] = edl_enquete['CD_PATRIM'].astype(str)

Suppression de la colonne DT_ALIM:

In [ ]:
edl_enquete.drop(columns='DT_ALIM', inplace=True)

Remplacement des Nan('str') en Valeur Nan:

In [ ]:
edl_enquete['PERS PHYSIQUE_MORALE'].replace('nan',np.nan, inplace=True)

Creation du DataFrame de test:

In [ ]:
test_edl_enquete = edl_enquete

Drop des colonnes non pertinente:

In [ ]:
test_edl_enquete.drop(columns=['ID_DOSCLIE', 'NO_DOSFACT', 'DTD_CTRAT', 'DTF_CTRAT', 'CD_PATRIM1', 'CD_PATRIM2', 'CD_PATRIM3', 'CD_PATRIM4', 'DEBUT_FRAIS_DT_SID_FACTURA_GIM', 
                    'FIN_FRAIS_DT_SID_FACTURA_GIM', 'CPT_FRAIS DOSS_SLS_EOS',	'MT_FRAIS DOSS_SLS_EOS'], inplace=True)

Remplacement des valeurs null(0) en Valeur Nan:

In [ ]:
test_edl_enquete['CPT_PENALITE RETARD_SLS_EOS'].replace(0, np.nan, inplace=True)

Drop des nan dans la colonnes cible:

In [ ]:
test_edl_enquete.dropna(axis=0, subset='CPT_PENALITE RETARD_SLS_EOS', inplace=True)

Changement du type de la colonnes cible:

In [ ]:
test_edl_enquete['CPT_PENALITE RETARD_SLS_EOS'] = test_edl_enquete['CPT_PENALITE RETARD_SLS_EOS'].astype(int)

Somme des penaliter de retard grouper par personne physique ou moral:

In [ ]:
test_edl_enquete.groupby('PERS PHYSIQUE_MORALE')['CPT_PENALITE RETARD_SLS_EOS'].sum()

Creation de la cle de jointure:

In [ ]:
test_edl_enquete['cle_jointure'] = test_edl_enquete[['CD_PATRIM', 'NO_CTRACLI']].astype(str).agg('-'.join, axis=1)

In [ ]:
test_edl_enquete.to_csv('test_edl_enquete.csv', index = False)

In [ ]:
test_edl_enquete


## EDL_patrimoine = Patrimoines des locataires sortant

In [ ]:
# Liste des colonnes à conserver
test_edl_patrimoine = edl_patrimoine[[
    'CD_PATRIM', 'ON_ZONTEND', 'DATE MISE EN GESTION', 'DATE FIN CONSTRUCTION', 'ZZ_AGEBAT_EDL', 'ZZ_ANCBAT_EDL', 
    'TYPOLOGIE', 'CATEG PLAFOND LOYER', 'NO_DEPT', 'LB_DEPT', 'ZZ_LB_TYCONSTR', 'ZZ_LB_TY_CNV', 'MT_LOYER', 
    'MT_CHARGES', 'MT_TOTRUB', 'SURF_HAB', 'ZZ_REPCHA_SH_SUR_3', 'ZZ_MT_LOYER_M2_SH', 'ZZ_MT_CHARGES_M2_SH', 
    'ZZ_MT_TOTRUB_M2_SH', 'NO_EDL', 'NO CONSTAT', 'LB_TYPEDL', 'DT_DOSEDL', 'DT_CONSEDL', 'NO_CTRACLI', 
    'NB_EDL_SORTANT_AVEC_HUISSIER', 'MT FACTURE AU LOCATAIRE (MT_INDEMREEL)', 'MT_FA_PRO', 'MT_FA_REC', 
    'MT_REELEDL_HT', 'MT_REELEDL_TTC', 'DUREE OCCUPATION MM', 'DUREE OCCUPATION AA', 'CD_PHYMOR', 'NB_UC_ENTREE', 
    'MT_RES_UC_ENTREE', 'MT_NIVRES_BR_ENTREE', 'NB_OCCUPAN_ENTREE', 'NB_PERCHA_ENTREE', 'NB_ENFANT_ENTREE', 
    'ON_JEUMENA_ENTREE', 'ZZ_AGE_CL_ENTREE', 'ON_SEULE_ENTREE', 'ON_FAMMONO_ENTREE', 'ON_DALO_ENTREE', 
    'CD_CPDALO_ENTREE', 'LB_CPDALO_ENTREE', 'ON_PDALPD_ENTREE', 'CD_CPDALPD_ENTREE', 'MT_PLAFOND_ENTREE', 
    'MT_RESSOU_ENTREE', 'CD_CTRATYP', 'LB_CTRATYP', 'TY_AID_ENT', 'NOM_AIDENT', 'LB_SITFAM_ACTUEL', 'OST_ACTUEL', 
    'NB_OCCUPAN_ACTUEL', 'NB_ENFANT _ACTUEL', 'AN_TOTREVI_ACTUEL', 'MT_TOTREVI_ACTUEL', 'ON_FAMMONO_ACTUEL', 
    'ON_JEUMENA_ACTUEL', 'MT_SOLDLOY_ACTUEL', 'MT_SOLD_DG_ACTUEL', 'NB_SIT_1000', 'NB_SIT_9180', 'NB_SIT_4500', 
    'NB_SIT_8200', 'NB_TYPSIT_Expulsion', 'NB_TYPSIT_Information', 'NB_TYPSIT_Contentieux', 'NB_TYPSIT_Action_sociale', 
    'MT_SOLD_LOY_A_FIN_CONTRAT', 'NB_ECHRES_A_FIN_CONTRAT', 'DATE_PREM_IMPAYE', 'MT_SOLD_DEB_PREM_IMPAYE', 
    'PLAGE AGE DETTE_PREM_IMPAYE'
]]




Affichage de la dimension du DatFrame:

In [ ]:
test_edl_patrimoine.shape

Affichage des type de colonnes du DataFrame:

In [ ]:
test_edl_patrimoine.info()

In [ ]:
test_edl_patrimoine[['DATE MISE EN GESTION', 'DATE FIN CONSTRUCTION', 'DT_DOSEDL', 'DT_CONSEDL', 'DATE_PREM_IMPAYE']]

Changement des colonnes cible en date P1:

In [ ]:
test_edl_patrimoine['DATE MISE EN GESTION'] = pd.to_datetime(test_edl_patrimoine['DATE MISE EN GESTION'])
test_edl_patrimoine['DATE FIN CONSTRUCTION'] = pd.to_datetime(test_edl_patrimoine['DATE FIN CONSTRUCTION'])

Changement des colonnes cible en date P2:

In [ ]:
test_edl_patrimoine['DT_DOSEDL'] = pd.to_datetime(test_edl_patrimoine['DT_DOSEDL'], format = 'mixed')
test_edl_patrimoine['DT_CONSEDL'] = pd.to_datetime(test_edl_patrimoine['DT_CONSEDL'], format = 'mixed')

Changement des colonnes cible en date P3:

In [ ]:
test_edl_patrimoine['DATE_PREM_IMPAYE'] = pd.to_datetime(test_edl_patrimoine['DATE_PREM_IMPAYE'], format = 'mixed')

Creation de la cle de jointure:

In [ ]:
test_edl_patrimoine['cle_jointure'] = test_edl_patrimoine[['CD_PATRIM', 'NO_CTRACLI']].astype(str).agg('-'.join, axis=1)

In [ ]:
test_edl_patrimoine['DUREE OCCUPATION MM'] = test_edl_patrimoine['DUREE OCCUPATION MM'].astype(pd.Int64Dtype())
test_edl_patrimoine['DUREE OCCUPATION AA'] = test_edl_patrimoine['DUREE OCCUPATION AA'].astype(pd.Int64Dtype())

Création du DataFrame pour les logements:

In [ ]:
patrimoine_logement = test_edl_patrimoine[['CD_PATRIM', 'ON_ZONTEND', 'ZZ_AGEBAT_EDL', 'TYPOLOGIE', 'LB_DEPT', 'ZZ_LB_TYCONSTR', 'ZZ_LB_TY_CNV', 'MT_TOTRUB', 'SURF_HAB', 'NO_EDL', 'MT_FA_REC', 'DUREE OCCUPATION MM', 'DUREE OCCUPATION AA', 'CD_PHYMOR', 'NB_PERCHA_ENTREE', 'ZZ_AGE_CL_ENTREE', 'ON_SEULE_ENTREE', 'ON_DALO_ENTREE', 'LB_CTRATYP', 'NOM_AIDENT', 'OST_ACTUEL', 'NB_OCCUPAN_ACTUEL', 'NB_ENFANT _ACTUEL', 'ON_FAMMONO_ACTUEL', 'ON_JEUMENA_ACTUEL', 'cle_jointure']]

Création du DataFrame pour les profils:

In [ ]:
patrimoine_profil = test_edl_patrimoine[[
    'CD_PATRIM',  
    'TYPOLOGIE', 'CATEG PLAFOND LOYER', 'LB_DEPT', 'ZZ_LB_TY_CNV', 'MT_LOYER', 
    'MT_TOTRUB',  
    'MT FACTURE AU LOCATAIRE (MT_INDEMREEL)', 'MT_FA_PRO', 'MT_FA_REC', 
    'DUREE OCCUPATION MM', 'DUREE OCCUPATION AA', 'CD_PHYMOR', 'ON_DALO_ENTREE', 
    'MT_PLAFOND_ENTREE', 
    'MT_RESSOU_ENTREE', 'NOM_AIDENT', 'LB_SITFAM_ACTUEL', 'OST_ACTUEL', 
    'NB_OCCUPAN_ACTUEL', 'NB_ENFANT _ACTUEL', 'AN_TOTREVI_ACTUEL', 'MT_TOTREVI_ACTUEL', 'ON_FAMMONO_ACTUEL', 
    'ON_JEUMENA_ACTUEL', 'MT_SOLDLOY_ACTUEL', 'MT_SOLD_DG_ACTUEL', 'NB_SIT_1000', 'NB_SIT_9180', 'NB_SIT_4500', 
    'NB_SIT_8200', 'NB_TYPSIT_Expulsion', 'NB_TYPSIT_Information', 'NB_TYPSIT_Contentieux', 'NB_TYPSIT_Action_sociale', 
    'MT_SOLD_LOY_A_FIN_CONTRAT', 'NB_ECHRES_A_FIN_CONTRAT', 'DATE_PREM_IMPAYE', 'MT_SOLD_DEB_PREM_IMPAYE', 
    'PLAGE AGE DETTE_PREM_IMPAYE', 'cle_jointure']]

## EDL_OST = table avec présence OST (Occupant Sans droit ni Titre)

Changement de type sur les colonnes cible:

In [ ]:
edl_ost['DTD_CTRAT'] = pd.to_datetime(edl_ost['DTD_CTRAT'])
edl_ost['DTF_CTRAT'] = pd.to_datetime(edl_ost['DTF_CTRAT'])
edl_ost['DT_ALIM'] = pd.to_datetime(edl_ost['DT_ALIM'])
edl_ost['CPT_OST'] = edl_ost['CPT_OST'].astype('bool') # peut-être utile de garder seulement cette ligne car les autres seront supprimées

Sauvegarde du DataFrame original et création d'un DataFrame de test:

In [ ]:
test_edl_ost = edl_ost.copy()

Drop des colonnes non pertinente: 

In [ ]:
test_edl_ost = test_edl_ost.drop(['ID_DOSCLIE', 'NO_DOSFACT', 
       'DTD_CTRAT', 'DTF_CTRAT', 'CD_PATRIM1', 'CD_PATRIM2',
       'CD_PATRIM3', 'CD_PATRIM4', 'NB_ECHEANCES_NON VALEUR',
       'MT_NON VALEUR', 'TRANCHE_MT_NON VALEUR', 'DT_ALIM'], axis = 1)
test_edl_ost

Creation de la cle de jointure:

In [ ]:
test_edl_ost['cle_jointure'] = test_edl_ost[['CD_PATRIM', 'NO_CTRACLI']].astype(str).agg('-'.join, axis=1)

## EDL_Travaux = Description du montant des travaux totaux par logement

Changement de type sur les colonnes cible:

In [ ]:
edl_travaux['DTD_CTRAT_MIN'] = pd.to_datetime(edl_travaux['DTD_CTRAT_MIN'])
edl_travaux['DT_ALIM'] = pd.to_datetime(edl_travaux['DT_ALIM'])
edl_travaux['DTD_CTRAT_MAX'] = pd.to_datetime(edl_travaux['DTD_CTRAT_MAX'], format= 'mixed') # peut-être utile de garder seulement cette ligne car les autres seront supprimées

Sauvegarde du DataFrame original et création d'un DataFrame de test:

In [ ]:
test_edl_travaux = edl_travaux.copy()

Drop des colonnes non pertinente: 

In [ ]:
test_edl_travaux = test_edl_travaux.drop(['NB_CTRATS_VAC', 'NB_CTRATS_LOC',
       'DTD_CTRAT_MIN', 'DTD_CTRAT_MAX',
       'MT_FAC_TTC', 'DT_ALIM'], axis=1)
test_edl_travaux

Pour cette table nous utiliserons CD_PATRIM comme cle de jointure.

## EDL_sollicitation = sollicitation d'intervention de type 'entretien' ou 'relationnel'

Changement de type sur les colonnes cible:

In [ ]:
edl_sollicitation['DTD_CTRAT'] = pd.to_datetime(edl_sollicitation['DTD_CTRAT'])
edl_sollicitation['DTF_CTRAT'] = pd.to_datetime(edl_sollicitation['DTF_CTRAT'])
edl_sollicitation['DT_ALIM'] = pd.to_datetime(edl_sollicitation['DT_ALIM'])
edl_sollicitation['DATE AFFAIRE'] = pd.to_datetime(edl_sollicitation['DATE AFFAIRE'])

Sauvegarde du DataFrame original et création d'un DataFrame de test:

In [ ]:
test_sollicitation = edl_sollicitation.copy()

Drop des colonnes non pertinente P1: 

In [ ]:
test_sollicitation = test_sollicitation.drop(['ID_DOSCLIE','NO_DOSFACT','CD_PATRIM1','CD_PATRIM2','CD_PATRIM3','CD_PATRIM4','CD_QUALIF1','CD_QUALIF2','CD_QUALIF3', 'DTD_CTRAT','DTF_CTRAT','DT_ALIM', 'CD_ETAT_AF'], axis=1)

Drop des colonnes non pertinente P2: 

In [ ]:
test_sollicitation = test_sollicitation.drop(['AFFAIRE_ENTRETIEN_ANNULEE', 'AFFAIRE_ENTRETIEN_REFUSEE','AFFAIRE_ENTRETIEN_ENCOURS', 'AFFAIRE_ENTRETIEN_SOLDEE', 'AFFAIRE_RELATIONNELLE_ANNULEE', 'AFFAIRE_RELATIONNELLE_REFUSEE','AFFAIRE_RELATIONNELLE_ENCOURS','AFFAIRE_RELATIONNELLE_SOLDEE','AFFAIRE_AUTRE'], axis=1)

Creation de la cle de jointure

In [ ]:
test_sollicitation['cle_jointure'] = test_sollicitation[['CD_PATRIM', 'NO_CTRACLI', 'NO_AFFAIRE']].astype(str).agg('-'.join, axis=1)

In [ ]:
test_sollicitation

In [ ]:
test_sollicitation['NO_AFFAIRE'] = test_sollicitation['NO_AFFAIRE'].drop_duplicates()

In [ ]:
test_sollicitation

In [ ]:
len(test_sollicitation['NO_AFFAIRE'].dropna())

## patrimoine_logement

## patrimoine_profil

# Creation de la table intermediaire

## table intermediaire

In [ ]:
test_sollicitation

In [ ]:
edl_entree_sortie

In [ ]:
Table1 = test_edl_es['cle_jointure']
Table2 = test_sollicitation['NO_AFFAIRE']

In [ ]:
Table_intermediaire = pd.concat([Table1, Table2], axis = 1)

In [ ]:
Table_intermediaire['cle_intermediaire'] = Table_intermediaire[['cle_jointure', 'NO_AFFAIRE']].astype(str).agg('-'.join, axis=1)

In [ ]:
Table_intermediaire['NO_AFFAIRE'].nunique()

In [ ]:
Table_intermediaire['cle_jointure'].isna().sum()

In [ ]:
table_intermediaire = Table_intermediaire.dropna()

In [ ]:
table_intermediaire

In [ ]:
table_intermediaire.to_csv('table_intermediaire.csv',index = False) 

In [ ]:
patrimoine_logement.head()

In [ ]:
patrimoine_logement.info()

In [ ]:
patrimoine_logement.columns

In [ ]:
patrimoine_profil

# Création des table Profil et Logement

## test de merge

In [ ]:
test_edl_enquete

In [ ]:
edl_entree_sortie

In [ ]:
test_de_merge = edl_entree_sortie.merge(test_edl_enquete, on = 'cle_jointure', how = 'left')

In [ ]:
test_de_merge2 = test_de_merge.merge(test_edl_ost, on = 'cle_jointure', how = 'left')

In [ ]:
test_de_merge3 = test_de_merge2.merge(test_edl_travaux, on = 'CD_PATRIM', how = 'left')

In [ ]:
test_de_merge3.columns

In [ ]:
test_de_merge3.drop(columns = ['CD_PATRIM_y', 'CD_PATRIM', 'NO_CTRACLI', 'PERS PHYSIQUE_MORALE_y'], axis = 1, inplace = True)

In [ ]:
test_de_merge3

In [ ]:
analyse_profil = test_de_merge3.merge(patrimoine_profil, on = 'cle_jointure', how = 'left')

In [ ]:
analyse_profil

In [ ]:
analyse_logement = test_de_merge3.merge(patrimoine_logement, on = 'cle_jointure', how = 'left')

In [ ]:
analyse_logement

In [ ]:
pyg.walk(analyse_profil)

## analyse_profil

In [ ]:
analyse_profil.columns

In [ ]:
analyse_profil = analyse_profil.drop(columns = ['NO_CTRACLI_y', 'CD_PATRIM'])

Re-definition des colonnes cible:

In [ ]:
analyse_profil.rename(columns = {'cle_primaire' : 'id_logement', 'CD_PATRIM_x' : 'CD_PATRIM', 'NO_CTRACLI_x' : 'NO_CTRACLI', 'PERS PHYSIQUE_MORALE_x' : 'PERS PHYSIQUE_MORALE'}, inplace = True)

In [ ]:
analyse_profil.columns

In [ ]:
analyse_profil = analyse_profil.reindex(['id_logement', 'cle_jointure', 'CD_PATRIM', 'NO_CTRACLI', 'DTD_CTRAT', 'DTF_CTRAT',
       'CD_TYPEDL_ENT', 'CD_TYPEDL_SOR', 'LB_NOTSANT_ENT', 'LB_NOTSANT_SOR',
       'PERS PHYSIQUE_MORALE', 'CPT_PENALITE RETARD_SLS_EOS',
       'MT_CPT_PENALITE RETARD_SLS_EOS', 'CPT_OST', 'NB_CTRATS', 'NB_FACTURES',
       'MT_FAC_HT', 'TYPOLOGIE', 'CATEG PLAFOND LOYER', 'LB_DEPT',
       'ZZ_LB_TY_CNV', 'MT_LOYER', 'MT_TOTRUB',
       'MT FACTURE AU LOCATAIRE (MT_INDEMREEL)', 'MT_FA_PRO', 'MT_FA_REC',
       'DUREE OCCUPATION MM', 'DUREE OCCUPATION AA', 'CD_PHYMOR',
       'ON_DALO_ENTREE', 'MT_PLAFOND_ENTREE', 'MT_RESSOU_ENTREE', 'NOM_AIDENT',
       'LB_SITFAM_ACTUEL', 'OST_ACTUEL', 'NB_OCCUPAN_ACTUEL',
       'NB_ENFANT _ACTUEL', 'AN_TOTREVI_ACTUEL', 'MT_TOTREVI_ACTUEL',
       'ON_FAMMONO_ACTUEL', 'ON_JEUMENA_ACTUEL', 'MT_SOLDLOY_ACTUEL',
       'MT_SOLD_DG_ACTUEL', 'NB_SIT_1000', 'NB_SIT_9180', 'NB_SIT_4500',
       'NB_SIT_8200', 'NB_TYPSIT_Expulsion', 'NB_TYPSIT_Information',
       'NB_TYPSIT_Contentieux', 'NB_TYPSIT_Action_sociale',
       'MT_SOLD_LOY_A_FIN_CONTRAT', 'NB_ECHRES_A_FIN_CONTRAT',
       'DATE_PREM_IMPAYE', 'MT_SOLD_DEB_PREM_IMPAYE',
       'PLAGE AGE DETTE_PREM_IMPAYE'], axis = 1)

In [ ]:
analyse_profil.info()

In [ ]:
analyse_profil.isna

In [ ]:
analyse_profil['NB_OCCUPAN_ACTUEL'].astype('Int64')
analyse_profil['NB_ENFANT _ACTUEL'].astype('Int64')
analyse_profil['NB_SIT_1000'].astype('Int64')
analyse_profil['NB_SIT_9180'].astype('Int64')
analyse_profil['NB_SIT_4500'].astype('Int64')
analyse_profil['NB_SIT_8200'].astype('Int64')
analyse_profil['NB_TYPSIT_Expulsion'].astype('Int64')
analyse_profil['NB_TYPSIT_Information'].astype('Int64')
analyse_profil['NB_TYPSIT_Contentieux'].astype('Int64')
analyse_profil['NB_TYPSIT_Action_sociale'].astype('Int64')
analyse_profil['NB_TYPSIT_Action_sociale'].astype('Int64')

In [ ]:
#analyse_profil.to_csv('analyse_profil.csv', index = False)

In [ ]:
analyse_profil['LB_SITFAM_ACTUEL'].value_counts()

## analyse_logement

In [ ]:
analyse_logement.head()

In [ ]:
analyse_logement.columns

In [ ]:
analyse_logement.drop(columns = ['NO_CTRACLI_y', 'CD_PATRIM'], inplace = True)

In [ ]:
analyse_logement.rename(columns = {'cle_primaire' : 'id_logement', 'CD_PATRIM_x' : 'CD_PATRIM', 'NO_CTRACLI_x' : 'NO_CTRACLI', 'PERS PHYSIQUE_MORALE_x' : 'PERS PHYSIQUE_MORALE'}, inplace = True)

In [ ]:
analyse_logement.columns

In [ ]:
analyse_logement = analyse_logement.reindex(['id_logement', 'cle_jointure', 'CD_PATRIM', 'NO_CTRACLI', 'DTD_CTRAT', 'DTF_CTRAT',
       'CD_TYPEDL_ENT', 'CD_TYPEDL_SOR', 'LB_NOTSANT_ENT', 'LB_NOTSANT_SOR',
       'PERS PHYSIQUE_MORALE', 'CPT_PENALITE RETARD_SLS_EOS',
       'MT_CPT_PENALITE RETARD_SLS_EOS', 'CPT_OST', 'NB_CTRATS', 'NB_FACTURES',
       'MT_FAC_HT', 'ON_ZONTEND', 'ZZ_AGEBAT_EDL', 'TYPOLOGIE', 'LB_DEPT',
       'ZZ_LB_TYCONSTR', 'ZZ_LB_TY_CNV', 'MT_TOTRUB', 'SURF_HAB', 'NO_EDL',
       'MT_FA_REC', 'DUREE OCCUPATION MM', 'DUREE OCCUPATION AA', 'CD_PHYMOR',
       'NB_PERCHA_ENTREE', 'ZZ_AGE_CL_ENTREE', 'ON_SEULE_ENTREE',
       'ON_DALO_ENTREE', 'LB_CTRATYP', 'NOM_AIDENT', 'OST_ACTUEL',
       'NB_OCCUPAN_ACTUEL', 'NB_ENFANT _ACTUEL', 'ON_FAMMONO_ACTUEL',
       'ON_JEUMENA_ACTUEL'], axis = 1)

In [ ]:
analyse_logement.head()

In [ ]:
analyse_logement.info()

# Feature engineering 

## Analyse logement (mise en place de la note de santé)

### Affichage du DataFrame:

In [ ]:
analyse_logement.head()

### Sauvegarde du DataFrame original:

In [ ]:
test_analyse_logement = analyse_logement.copy()

### Mise en place de la note globale:

In [ ]:
test_analyse_logement['etat_de_sante'] = test_analyse_logement['LB_NOTSANT_SOR'] - test_analyse_logement['LB_NOTSANT_ENT']

In [ ]:
test_analyse_logement['etat_de_sante'].describe()

### Mise en place du barem

In [ ]:
def palier_sante(x):
        if x == 0:
            return 'A'
        elif x > 0:
            return 'A+'
        elif 0 > x >= -100:
            return 'B'
        elif -100 > x >= -400:
            return 'C'
        else:
            return 'D'

In [ ]:
test_analyse_logement['separateur_sante'] = test_analyse_logement['etat_de_sante'].apply(palier_sante)

In [ ]:
test_analyse_logement.columns

In [ ]:
test_analyse_logement['OST_ACTUEL'].fillna(0, inplace = True)

In [ ]:
test_analyse_logement['OST_ACTUEL'] = test_analyse_logement['OST_ACTUEL'].replace(to_replace='OST', value=1)

In [ ]:
test_analyse_logement['OST_ACTUEL'] = test_analyse_logement['OST_ACTUEL'].astype('int64')

In [ ]:
test_analyse_logement['OST_ACTUEL'].unique()

In [ ]:
piedata = test_analyse_logement.groupby(['OST_ACTUEL'])['etat_de_sante'].mean()
piedata

In [ ]:
test_analyse_logement['separateur_sante'].unique()

### export du DataFrame en CSV:

In [ ]:
# test_analyse_logement.to_csv('analyse_logement.csv', index = False)

Affichage du DataFrame final:

In [ ]:
test_analyse_logement

## Analyse_profil (mise en place du coefficient)

### Mise en place du DataFrame de test:

In [ ]:
analyse_profil.columns

In [ ]:
test_analyse_profil = analyse_profil[['id_logement', 'OST_ACTUEL', 'NB_OCCUPAN_ACTUEL', 'LB_SITFAM_ACTUEL', 'ON_DALO_ENTREE', 'CPT_PENALITE RETARD_SLS_EOS','NB_SIT_1000', 'NB_SIT_9180', 'NB_SIT_4500', 'NB_SIT_8200']]

### Remplacement des Nan par des 0:

In [ ]:
test_analyse_profil['OST_ACTUEL'].fillna(0, inplace = True)

### Remplacement OST par 1 et changement du type:

In [ ]:
test_analyse_profil['OST_ACTUEL'] = test_analyse_profil['OST_ACTUEL'].replace(to_replace='OST', value=1)

In [ ]:
test_analyse_profil['OST_ACTUEL'] = test_analyse_profil['OST_ACTUEL'].astype('int64')

In [ ]:
test_analyse_profil.info()

In [ ]:
analyse_profil['CPT_OST'].value_counts()

In [ ]:
test_analyse_profil['OST_ACTUEL'].value_counts()

### Mise en place du barem:

In [ ]:
test_analyse_profil['val_ost'] = test_analyse_profil['OST_ACTUEL'] * 3

In [ ]:
one_hot = pd.get_dummies(test_analyse_profil['LB_SITFAM_ACTUEL'])


In [ ]:
pd.concat([test_analyse_profil['LB_SITFAM_ACTUEL'], one_hot], axis=1)

In [ ]:
test_analyse_profil[['sit_fam','val_celib', 'val_concubinage', 'val_divorce', 'val_marie(e)', 'val_pacse', 'val_pm', 'val_separe', 'val_veuf']] = pd.concat([test_analyse_profil['LB_SITFAM_ACTUEL'], one_hot], axis=1)

In [ ]:
test_analyse_profil['CPT_PENALITE RETARD_SLS_EOS'] = test_analyse_profil['CPT_PENALITE RETARD_SLS_EOS'].astype('Int64')

In [ ]:
test_analyse_profil['CPT_PENALITE RETARD_SLS_EOS'] = test_analyse_profil['CPT_PENALITE RETARD_SLS_EOS'].fillna(0)

In [ ]:
test_analyse_profil.info()

In [ ]:
def val_pnt_rtd(x):
        if x == 0:
            return 0
        elif 1 <= x <= 2:
            return 1
        elif 2 < x < 5:
            return 2
        elif 5 <= x < 11:
            return 3
        else:
            return 4

In [ ]:
test_analyse_profil['val_ptn_rtd'] = test_analyse_profil['CPT_PENALITE RETARD_SLS_EOS'].apply(val_pnt_rtd)

In [ ]:
test_analyse_profil['ON_DALO_ENTREE'] = test_analyse_profil['ON_DALO_ENTREE'].replace({'N': 0, 'O': 1})

In [ ]:
test_analyse_profil['ON_DALO_ENTREE'] = test_analyse_profil['ON_DALO_ENTREE'].astype('Int64')

In [ ]:
test_analyse_profil['val_dalo'] = test_analyse_profil['ON_DALO_ENTREE'] * 1.5

In [ ]:
test_analyse_profil['val_dalo'] = test_analyse_profil['val_dalo'].astype('Int64')

In [ ]:
def bareme_occupants(x): 
    if x <2: 
        return 1
    elif 2<= x <4: 
        return 1.5
    else:
        return 2

In [ ]:
test_analyse_profil['bareme_occupants'] = test_analyse_profil['NB_OCCUPAN_ACTUEL'].apply(bareme_occupants)

In [ ]:
test_analyse_profil['bareme_occupants'] = test_analyse_profil['bareme_occupants'].astype('int64')

In [ ]:
test_analyse_profil[['NB_SIT_1000', 'NB_SIT_9180', 'NB_SIT_4500', 'NB_SIT_8200']] = test_analyse_profil[['NB_SIT_1000', 'NB_SIT_9180', 'NB_SIT_4500', 'NB_SIT_8200']].astype('Int64')

In [ ]:
test_analyse_profil['val_sit_1000'] = test_analyse_profil['NB_SIT_1000']

In [ ]:
test_analyse_profil['val_sit_9180'] = test_analyse_profil['NB_SIT_9180'] * 3

In [ ]:
test_analyse_profil['val_sit_4500'] = test_analyse_profil['NB_SIT_4500'] * 4

In [ ]:
test_analyse_profil['val_sit_8200'] = test_analyse_profil['NB_SIT_8200'] * 2

In [ ]:
test_analyse_profil.columns

In [ ]:
test_analyse_profil[['val_celib', 'val_concubinage', 'val_divorce', 'val_marie(e)', 'val_pacse', 'val_pm', 'val_separe', 'val_veuf']] = test_analyse_profil[[
       'val_celib', 'val_concubinage', 'val_divorce', 'val_marie(e)',
       'val_pacse', 'val_pm', 'val_separe', 'val_veuf']].astype('Int64')

In [ ]:
test_analyse_profil['etat_de_profil'] = test_analyse_profil['val_ost'] + test_analyse_profil['val_celib'] + test_analyse_profil['val_concubinage'] + test_analyse_profil['val_divorce'] + test_analyse_profil['val_marie(e)'] + test_analyse_profil['val_pacse'] + test_analyse_profil['val_pm'] + test_analyse_profil['val_separe'] + test_analyse_profil['val_veuf'] + test_analyse_profil['val_ptn_rtd'] + test_analyse_profil['val_dalo'] + test_analyse_profil['bareme_occupants'] + test_analyse_profil['val_sit_1000'] + test_analyse_profil['val_sit_9180'] + test_analyse_profil['val_sit_4500'] + test_analyse_profil['val_sit_8200']

In [ ]:
test_analyse_profil_final = test_analyse_profil

In [ ]:
test_analyse_profil_final.drop(columns = ['val_ost', 'sit_fam',
       'val_celib', 'val_concubinage', 'val_divorce', 'val_marie(e)',
       'val_pacse', 'val_pm', 'val_separe', 'val_veuf', 'val_ptn_rtd',
       'val_dalo', 'bareme_occupants', 'val_sit_1000', 'val_sit_9180',
       'val_sit_4500', 'val_sit_8200'], axis = 1, inplace=True)

In [ ]:
test_analyse_profil_final

### Jointure avec le DataFrame originale:

In [ ]:
analyse_profil = analyse_profil.merge(test_analyse_profil, on = 'id_logement', how = 'left')

In [ ]:
analyse_profil['etat_de_sante'] = analyse_profil['LB_NOTSANT_SOR'] - analyse_profil['LB_NOTSANT_ENT']

### Mise en place de la note santé:

In [ ]:
def palier_sante(x):
        if x == 0:
            return 'A'
        elif x > 0:
            return 'A+'
        elif 0 > x >= -100:
            return 'B'
        elif -100 > x >= -400:
            return 'C'
        else:
            return 'D'

In [ ]:
analyse_profil['separateur_sante'] = analyse_profil['etat_de_sante'].apply(palier_sante)

In [ ]:
analyse_profil.drop(columns = ['OST_ACTUEL_y',
       'NB_OCCUPAN_ACTUEL_y', 'LB_SITFAM_ACTUEL_y', 'ON_DALO_ENTREE_y',
       'CPT_PENALITE RETARD_SLS_EOS_y', 'NB_SIT_1000_y', 'NB_SIT_9180_y',
       'NB_SIT_4500_y', 'NB_SIT_8200_y'], axis=1, inplace = True)

In [ ]:
analyse_profil['etat_de_profil'].value_counts()

In [ ]:
analyse_profil['etat_de_profil'].describe()

### Définition du profil a risque:

In [ ]:
def palier_profil(x):
        if pd.isna(x):
            return np.nan
        elif x <= 3:
            return 'Très_Faible'
        elif 4 <= x <= 5:
            return 'Faible'
        elif 6 <= x <= 15:
            return 'Moyen'
        elif 16 <= x <= 30:
            return 'Elevé'
        else:
            return 'Très_Elevé'

In [ ]:
analyse_profil['profil_a_risque'] = analyse_profil['etat_de_profil'].apply(palier_profil)

In [ ]:
analyse_profil['profil_a_risque'].value_counts()

### Test de correlation avec le separateur sante et le profil a risque:

In [ ]:
analyse_profil.columns

In [ ]:
heatmap_test = analyse_profil[['separateur_sante', 'profil_a_risque']]

In [ ]:
heatmap_test['separateur_sante'].replace({'A+' : 1, 'A' : 2, 'B' : 3, 'C' : 4, 'D' : 5}, inplace = True)

In [ ]:
heatmap_test['profil_a_risque'].replace({'Très_Faible' : 1, 'Faible' : 2, 'Moyen' : 3, 'Elevé' : 4, 'Très_Elevé' : 5}, inplace = True)

In [ ]:
corrmat = heatmap_test.corr()

In [ ]:
sns.heatmap(corrmat, annot = True)

### Analyse de correlation possible:

In [ ]:
test_Nico_amour = pd.DataFrame(analyse_profil.groupby(['separateur_sante','profil_a_risque'])['profil_a_risque'].count())

In [ ]:
test_Nico_amour

In [ ]:
analyse_profil.loc[analyse_profil['profil_a_risque'] == 'Très_Faible']

In [ ]:
test_Fanny_amour = pd.DataFrame(analyse_profil.groupby(['CD_TYPEDL_SOR','profil_a_risque'])['profil_a_risque'].count())

In [ ]:
test_Fanny_amour

In [ ]:
analyse_profil

In [ ]:
test_Jessica_amour = pd.DataFrame(analyse_profil.groupby(['MT_FA_REC'])['MT_FA_REC'].count())

In [ ]:
test_Jessica_amour

In [ ]:
test_Thibault_amour = pd.DataFrame(analyse_profil.groupby(['MT_FA_REC', 'separateur_sante', 'profil_a_risque'])['profil_a_risque'].count())

In [ ]:
test_Thibault_amour

In [ ]:
test_Thibault_amour2 = pd.DataFrame(analyse_profil.groupby('NB_OCCUPAN_ACTUEL_x').agg(es=('etat_de_sante', 'mean'), co=('NB_OCCUPAN_ACTUEL_x', 'count')).reset_index())

In [ ]:
test_Thibault_amour2

In [ ]:
test_Thibault_amour3 = pd.DataFrame(analyse_profil.groupby('DUREE OCCUPATION AA').agg(es=('etat_de_sante', 'mean'), co=('DUREE OCCUPATION AA', 'count')).reset_index())

In [ ]:
test_Thibault_amour3

In [ ]:
df_temp= analyse_profil.groupby('NB_OCCUPAN_ACTUEL_x').agg( moyenne_es=('etat_de_sante', 'mean'), count_nb=('NB_OCCUPAN_ACTUEL_x', 'count') ).reset_index()
moyenne_generale = analyse_profil['etat_de_sante'].mean()
df_temp['ecart_en_pourcentage'] = ( (df_temp['moyenne_es'] - moyenne_generale) / moyenne_generale * 100)
df_temp

In [ ]:
moyenne_generale

In [ ]:
df_temp= analyse_profil.groupby('NB_OCCUPAN_ACTUEL_x').agg( moyenne_es=('etat_de_sante', 'mean'), count_nb=('NB_OCCUPAN_ACTUEL_x', 'count') ).reset_index()
df_temp = df_temp.sort_values(by = 'NB_OCCUPAN_ACTUEL_x')
df_temp['ecart_en_pourcentage'] = df_temp['moyenne_es'].pct_change()*100
df_temp

In [ ]:
df_temp_2= analyse_profil.groupby('DUREE OCCUPATION AA').agg( moyenne_es_2=('etat_de_sante', 'mean'), count_nb_2=('DUREE OCCUPATION AA', 'count') ).reset_index()
df_temp_2 = df_temp_2.sort_values(by = 'DUREE OCCUPATION AA')
df_temp_2['ecart_en_pourcentage'] = df_temp_2['moyenne_es_2'].pct_change()*100
df_temp_2

In [ ]:
dt = pd.DataFrame(analyse_profil[analyse_profil['MT_FA_REC']!=0].describe())

In [ ]:
dt['MT_FA_REC']

In [ ]:
analyse_finale = test_analyse_logement.merge(analyse_profil[['id_logement','profil_a_risque', 'MT_SOLD_LOY_A_FIN_CONTRAT','NB_ECHRES_A_FIN_CONTRAT', 'DATE_PREM_IMPAYE','MT_SOLD_DEB_PREM_IMPAYE', 'PLAGE AGE DETTE_PREM_IMPAYE', 'LB_SITFAM_ACTUEL_x']], on = 'id_logement', how = 'left')

In [ ]:
analyse_profil.columns

In [ ]:
analyse_finale.columns

In [ ]:
# analyse_finale.to_csv('analyse_finale.csv', index = False)

# Machine Learning

## ML Logement

In [ ]:
test_ML = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\ML\analyse_finaleml.csv")

In [ ]:
'CPT_OST','ON_ZONTEND', 'ON_DALO_ENTREE', 'LB_CTRATYP', 'SURF_HAB', 'TYPOLOGIE','ON_SEULE_ENTREE'

In [ ]:
df = test_ML.dropna(subset=['PERS PHYSIQUE_MORALE'])
colonne_to_drop = ['id_logement', 'cle_jointure', 'CD_PATRIM','DTD_CTRAT','DTF_CTRAT','CD_TYPEDL_ENT','CD_TYPEDL_SOR', 'NB_CTRATS', 'NB_FACTURES', 'MT_FAC_HT','LB_DEPT',
'ZZ_LB_TYCONSTR','ZZ_LB_TY_CNV','MT_TOTRUB', 'NO_EDL', 'MT_FA_REC', 'CD_PHYMOR', 'ZZ_AGE_CL_ENTREE', 'NOM_AIDENT', 'separateur_sante',
'MT_SOLD_LOY_A_FIN_CONTRAT','NB_ECHRES_A_FIN_CONTRAT','DATE_PREM_IMPAYE','MT_SOLD_DEB_PREM_IMPAYE','PLAGE AGE DETTE_PREM_IMPAYE', 'etat_de_sante', 'DUREE OCCUPATION MM', 'DUREE OCCUPATION AA', 'CPT_OST']
df = df.drop(colonne_to_drop, axis=1)
df = df.drop(columns='NB_PERCHA_ENTREE', axis=1)
df = df.dropna(subset=['LB_SITFAM_ACTUEL_x'])
df = df.dropna(subset=['profil_a_risque'])
df = df.dropna(subset=['etat_de_profil'])
df = df.drop(columns='PERS PHYSIQUE_MORALE', axis=1)
df['LB_SITFAM_ACTUEL_x'].unique()
df_temp = pd.get_dummies(df, columns=['LB_SITFAM_ACTUEL_x'])
df_temp = pd.get_dummies(df_temp, columns=['ON_FAMMONO_ACTUEL'])
df_temp = pd.get_dummies(df_temp, columns=['ON_JEUMENA_ACTUEL'])
df_temp = pd.get_dummies(df_temp, columns=['profil_a_risque'])
df_temp = pd.get_dummies(df_temp, columns=['ON_ZONTEND'])
df_temp = pd.get_dummies(df_temp, columns=['ON_DALO_ENTREE'])
df_temp = pd.get_dummies(df_temp, columns=['LB_CTRATYP'])
df_temp = pd.get_dummies(df_temp, columns=['TYPOLOGIE'])
df_temp = pd.get_dummies(df_temp, columns=['ON_SEULE_ENTREE'])
df_temp = pd.get_dummies(df_temp, columns=['OST_ACTUEL'])
df_temp.info()

In [ ]:
df_temp['LB_NOTSANT_SOR'].mean()

In [ ]:
# Préparer les données : convertir toutes les colonnes en type numérique si possible
df_numeric = df_temp.apply(pd.to_numeric, errors='coerce')
# Calculer la matrice de corrélation
corr_matrix = df_numeric.corr()
# Configurer la taille de la figure pour la heatmap
plt.figure(figsize=(12, 10))
# Générer la heatmap avec seaborn
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# Ajouter le titre
plt.title('Heatmap des corrélations du DataFrame', fontsize=16)
# Afficher la heatmap
plt.show()

In [ ]:
# Division des données en ensembles d'entraînement et de test
X = df_temp.drop('LB_NOTSANT_SOR', axis=1)
y = df_temp['LB_NOTSANT_SOR']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle de forêt aléatoire initial
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Importance des caractéristiques
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Sélection des caractéristiques les plus importantes
sfm = SelectFromModel(rf_model, threshold=0.01)
sfm.fit(X_train, y_train)

X_train_important = sfm.transform(X_train)
X_test_important = sfm.transform(X_test)

# Nouvelle recherche de grille avec les caractéristiques importantes
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_important, y_train)
print("Best parameters found:", grid_search.best_params_)

# Modèle avec les meilleurs paramètres
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(X_test_important)
mse_best = mean_squared_error(y_test, y_pred_best, squared=False)
r2_best = r2_score(y_test, y_pred_best)
print(f"Best Model - Mean Squared Error (MSE): {mse_best}")
print(f"Best Model - R-squared (R2): {r2_best}")

# Appliquer le modèle optimisé au dataset de base
df_temp['Predictions'] = best_rf_model.predict(sfm.transform(df_temp[X.columns]))
print(df_temp[['LB_NOTSANT_SOR', 'Predictions']])


In [ ]:
pct_erreur = (mse_best / df_temp['LB_NOTSANT_SOR'].mean())*100
print(f"Le pourcentage d'erreur de notre modéle est de : {pct_erreur} %")

## ML Profil

In [ ]:
ML_profil = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\VS Code\analyse_profil.csv")

In [ ]:
analysefinale = pd.read_csv(r"C:\Users\thiba\Documents\Projet_Final\ML\analyse_finaleml.csv")

In [ ]:
ML_profil = ML_profil.merge(analysefinale[['profil_a_risque', 'etat_de_profil', 'cle_jointure']], on='cle_jointure', how='left')

In [ ]:
ML_profil.info()

In [ ]:
ML_profil.columns

In [ ]:
df_profil = ML_profil
colonne_to_drop_1 = ['id_logement', 'cle_jointure', 'CD_PATRIM','DTD_CTRAT',  'PERS PHYSIQUE_MORALE',
       'DTF_CTRAT', 'MT_CPT_PENALITE RETARD_SLS_EOS', 'CPT_OST', 'NB_CTRATS', 'NB_FACTURES',
       'MT_FAC_HT', 'TYPOLOGIE', 'CATEG PLAFOND LOYER', 'LB_DEPT',
       'ZZ_LB_TY_CNV','MT_TOTRUB',
       'MT FACTURE AU LOCATAIRE (MT_INDEMREEL)', 'MT_FA_PRO', 'CD_PHYMOR',  'MT_PLAFOND_ENTREE', 'MT_RESSOU_ENTREE', 'NOM_AIDENT', 'AN_TOTREVI_ACTUEL', 'MT_TOTREVI_ACTUEL',
       'MT_SOLDLOY_ACTUEL',
       'MT_SOLD_DG_ACTUEL', 'NB_ECHRES_A_FIN_CONTRAT',
       'DATE_PREM_IMPAYE', 'MT_SOLD_DEB_PREM_IMPAYE',
       'PLAGE AGE DETTE_PREM_IMPAYE', 'profil_a_risque']
df_profil = df_profil.drop(colonne_to_drop_1, axis=1)

In [ ]:
df_profil['OST_ACTUEL'].replace({np.nan : False, 'OST' : True}, inplace=True)
df_profil['CPT_PENALITE RETARD_SLS_EOS'].replace({np.nan : 0}, inplace=True)

In [ ]:
df_profil.dropna(inplace=True)

In [ ]:
df_profil['OST_ACTUEL'].unique()

In [ ]:
df_profil.info()

In [ ]:
df_profil['MT_SOLD_LOY_A_FIN_CONTRAT'].replace(',', '.', regex=True, inplace=True)

In [ ]:
df_profil['MT_SOLD_LOY_A_FIN_CONTRAT'] = df_profil['MT_SOLD_LOY_A_FIN_CONTRAT'].astype('float64')

In [ ]:
df_profil_temp = pd.get_dummies(df_profil, columns=['CD_TYPEDL_ENT'])
df_profil_temp = pd.get_dummies(df_profil_temp, columns=['CD_TYPEDL_SOR'])
df_profil_temp = pd.get_dummies(df_profil_temp, columns=['ON_DALO_ENTREE'])
df_profil_temp = pd.get_dummies(df_profil_temp, columns=['ON_FAMMONO_ACTUEL'])
df_profil_temp = pd.get_dummies(df_profil_temp, columns=['ON_JEUMENA_ACTUEL'])
df_profil_temp = pd.get_dummies(df_profil_temp, columns=['OST_ACTUEL'])
df_profil_temp = pd.get_dummies(df_profil_temp, columns=['LB_SITFAM_ACTUEL'])
df_profil_temp.info()

In [ ]:
# Préparer les données : convertir toutes les colonnes en type numérique si possible
df_numeric_profil = df_profil_temp.apply(pd.to_numeric, errors='coerce')
# Calculer la matrice de corrélation
corr_matrix_profil = df_numeric_profil.corr()
# Configurer la taille de la figure pour la heatmap
plt.figure(figsize=(12, 10))
# Générer la heatmap avec seaborn
sns.heatmap(corr_matrix_profil, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# Ajouter le titre
plt.title('Heatmap des corrélations du DataFrame', fontsize=16)
# Afficher la heatmap
plt.show()

In [ ]:
# Division des données en ensembles d'entraînement et de test
X = df_profil_temp.drop('etat_de_profil', axis=1)
y = df_profil_temp['etat_de_profil']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle de forêt aléatoire initial
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Importance des caractéristiques
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Sélection des caractéristiques les plus importantes
sfm = SelectFromModel(rf_model, threshold=0.01)
sfm.fit(X_train, y_train)

X_train_important = sfm.transform(X_train)
X_test_important = sfm.transform(X_test)

# Nouvelle recherche de grille avec les caractéristiques importantes
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_important, y_train)
print("Best parameters found:", grid_search.best_params_)

# Modèle avec les meilleurs paramètres
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(X_test_important)
r_mse_best = mean_squared_error(y_test, y_pred_best, squared=False)
r2_best = r2_score(y_test, y_pred_best)
print(f"Best Model - Mean Squared Error (MSE): {r_mse_best}")
print(f"Best Model - R-squared (R2): {r2_best}")

# Appliquer le modèle optimisé au dataset de base
df_profil_temp['Predictions'] = best_rf_model.predict(sfm.transform(df_profil_temp[X.columns]))
print(df_profil_temp[['etat_de_profil', 'Predictions']])

In [ ]:
df_profil_temp['etat_de_profil'].mean()

In [ ]:
pct_erreur = (r_mse_best / df_profil_temp['etat_de_profil'].mean())*100
print(f"Le pourcentage d'erreur de notre modéle est de : {pct_erreur} %")